In [3]:
from pyss3 import SS3
from pyss3.util import Dataset
from pyss3.server import Server
import pandas as pd 
import numpy as np

from sklearn.metrics import accuracy_score

/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [29]:
import glob 
review_file = "/Users/jakubmojsiejuk/Documents/agh/filmweb-nlp/reviews_for_bert*.csv"

reviews = pd.DataFrame()
for rev_file in glob.glob(review_file):
    r = pd.read_csv(rev_file)
    reviews = pd.concat([reviews, r])
    
msk = np.random.rand(len(reviews)) < 0.7
reviews['sent'] = reviews['rating'].apply(lambda x: 'pos' if x > 5 else 'neg')
X_train, y_train = reviews['content'][msk].tolist(), reviews['sent'][msk].tolist()
X_test, y_test = reviews['content'][~msk].tolist(), reviews['sent'][~msk].tolist()

In [26]:
reviews['sent'] = (reviews['rating'] > 5)
s = np.sum(reviews['sent'])

print(s*100/len(reviews), len(reviews))

66.89828218490639 5181


In [8]:
clf = SS3()
s, l, p, _ = clf.get_hyperparameters()

print("Smoothness(s):", s)
print("Significance(l):", l)
print("Sanction(p):", p)

Smoothness(s): 0.45
Significance(l): 0.5
Sanction(p): 1


In [30]:
clf.fit(X_train, y_train)

 Training: 100%|██████████| 2/2 [00:15<00:00,  7.73s/it]


In [32]:
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_pred, y_test))

 Classification: 100%|██████████| 1540/1540 [00:25<00:00, 60.16it/s]

Accuracy: 0.7337662337662337


In [33]:
clf.fit(X_train, y_train, n_grams=3) 
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_pred, y_test))

 Classification: 100%|██████████| 1540/1540 [00:28<00:00, 53.33it/s]

Accuracy: 0.7350649350649351


In [34]:
for ngram in range(1, 5):
    clf.fit(X_train, y_train, n_grams=ngram) 
    y_pred = clf.predict(X_test)
    print("Accuracy:", accuracy_score(y_pred, y_test))

 Training:   0%|          | 0/2 [00:00<?, ?it/s]

Accuracy: 0.7350649350649351


 Training:   0%|          | 0/2 [00:00<?, ?it/s]

Accuracy: 0.7357142857142858


 Training:   0%|          | 0/2 [00:00<?, ?it/s]

Accuracy: 0.7357142857142858


 Classification: 100%|██████████| 1540/1540 [00:26<00:00, 57.23it/s]

Accuracy: 0.7357142857142858


In [48]:
# more radical reviews
radical_reviews = reviews.loc[(reviews['rating'] <= 3) | (reviews['rating'] >= 8)]
msk = np.random.rand(len(radical_reviews)) < 0.7
radical_reviews['sent'] = radical_reviews['rating'].apply(lambda x: 'pos' if x > 5 else 'neg')
X_train, y_train = radical_reviews['content'][msk].tolist(), radical_reviews['sent'][msk].tolist()
X_test, y_test = radical_reviews['content'][~msk].tolist(), radical_reviews['sent'][~msk].tolist()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [56]:
clf.set_hyperparameters(s=.44, l=.48, p=1.1)
for ngram in range(1, 5):
    clf.fit(X_train, y_train, n_grams=ngram) 
    y_pred = clf.predict(X_test)
    print("Accuracy:", accuracy_score(y_pred, y_test))

 Training:   0%|          | 0/2 [00:00<?, ?it/s]

Accuracy: 0.8086124401913876


 Training:   0%|          | 0/2 [00:00<?, ?it/s]

Accuracy: 0.8181818181818182


 Training:   0%|          | 0/2 [00:00<?, ?it/s]

Accuracy: 0.8181818181818182


 Classification: 100%|██████████| 418/418 [00:07<00:00, 53.86it/s]

Accuracy: 0.8205741626794258


In [54]:
radical_reviews['rating'].value_counts()

8     617
3     386
9     152
2     145
1      32
10     26
Name: rating, dtype: int64